In [72]:
import csv
import numpy as np

In [73]:
with open('data-2005/game.csv', 'r') as f:
  reader = csv.reader(f)
  Game = list(reader)


with open('data-2005/game-statistics.csv', 'r') as f:
  reader = csv.reader(f)
  GameStatistics = list(reader)
    
with open('data-2005/team-game-statistics.csv', 'r') as f:
  reader = csv.reader(f)
  TeamGameStatistics = list(reader)

In [74]:
print(len(Game))
Game = Game[1:]
TeamGameStatistics = TeamGameStatistics[1:]
print(len(Game))

719
718


In [75]:
print(Game[0])
#TeamGameStatistics[100]
print(len(TeamGameStatistics))

['0086016420050901', '09/01/2005', '86', '164', '25', 'TEAM']
1436


In [76]:
TeamGameStatistics[0]

['5',
 '0005055920050910',
 '21',
 '23',
 '0',
 '46',
 '26',
 '362',
 '3',
 '0',
 '0',
 '5',
 '144',
 '0',
 '2',
 '-2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '3',
 '3',
 '0',
 '0',
 '0',
 '0',
 '0',
 '24',
 '7',
 '260',
 '5',
 '228',
 '0',
 '0',
 '0',
 '4',
 '2',
 '59',
 '18',
 '2.0',
 '4',
 '0.0',
 '0',
 '0',
 '1',
 '2',
 '0',
 '4',
 '12',
 '1',
 '1433',
 '5',
 '42',
 '13',
 '2',
 '3',
 '0',
 '2',
 '1',
 '1']

In [77]:
games = {}
games.update({'gameID':0})
games.update({'date':0})
games.update({'attendance':0})
games.update({'duration':0})

In [78]:
games

{'attendance': 0, 'date': 0, 'duration': 0, 'gameID': 0}

In [79]:
#Assuming no anomolies in game data in TEAMGAMESTATS correspnoding to games in GAME
#Add fields from TEAMSGAMESTATS in this function
def SearchGame(GameID, TeamGameStatistics, HomeTeam, VisitTeam):
    count = 0
    for i,gamestats in enumerate(TeamGameStatistics):
        if(gamestats[1] == GameID):
            if(gamestats[0] == HomeTeam):
                HomeTeamStats = gamestats
            else:
                VisitTeamStats = gamestats
            count = count + 1
        if(count == 2):
            return(HomeTeamStats,VisitTeamStats)
    print("Loop ended without returning! :( ")

In [80]:
def listOfDictionaries(GameList, Game, GameStatistics, TeamGameStatistics):
    newGame = {}
    newGame['Date'] = Game[1]
    newGame['GameID'] = Game[0]
    newGame['Attendance'] = GameStatistics[1]
    newGame['Duration'] = GameStatistics[2]
    newGame['HomeTeam'] = int(Game[3])
    newGame['VisitTeam'] = int(Game[2])
    HomeTeamStats, VisitTeamStats = SearchGame(Game[0],TeamGameStatistics,Game[3],Game[2])
    newGame['HT-Points'] = int(HomeTeamStats[35])
    newGame['VT-Points'] = int(VisitTeamStats[35])
    GameList.append(newGame)
    return GameList

In [81]:
GameList = [] #List of dictionaries, based on sorted order of games
TeamList = [] #List of Teams in sorted order of Team codes
count = 0
for i in range(len(Game)):
    GameList = listOfDictionaries(GameList, Game[i], GameStatistics[i], TeamGameStatistics)
    if(GameList[i]['HomeTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['HomeTeam'])
    if(GameList[i]['VisitTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['VisitTeam'])
        
TeamList.sort()

In [82]:
GameList[100]

{'Attendance': '49276',
 'Date': '09/10/2005',
 'Duration': '178',
 'GameID': '0648025720050910',
 'HT-Points': 17,
 'HomeTeam': 257,
 'VT-Points': 15,
 'VisitTeam': 648}

In [83]:
NumberOfWins = []
NumberOfMatches = []
for i,team in enumerate(TeamList):
    NumberOfWins.append(0)
    NumberOfMatches.append(0)

In [84]:
len(TeamList)

166

In [85]:
X_train = []
X_test = []
Y_train = []
Y_test = []

In [86]:
#Skip first 100 games
for i,game in enumerate(GameList[:100]):
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    if(game['HT-Points'] >= game['VT-Points']):
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1        
    

In [87]:
for i,game in enumerate(GameList[100:]):
    temp = []
   # print(i)
    #print(game)
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    if(NumberOfMatches[htindex] > 0):
        temp.append(NumberOfWins[htindex]/NumberOfMatches[htindex])
    else:
        temp.append(0.0)
    if(NumberOfMatches[vtindex] > 0):
        temp.append(NumberOfWins[vtindex]/NumberOfMatches[vtindex])
    else:
        temp.append(0.0)
    X_train.append(temp)
    if(game['HT-Points'] >= game['VT-Points']):
        Y_train.append(1)
    else:
        Y_train.append(0)

In [88]:
len(X_train)

618

In [89]:
Y_train[100]

1